In [ ]:
import argparse
import collections
import struct
import sys
import serial
import socket
from time import sleep

In [ ]:
class GBOperation(object):
    OPERATION_HEADER_FMT        = '<HHBBBB'

    def __init__(self, _type, _cport, _id = 0, local = 0):
        self.type = _type
        self.id = _id
        self.result = 0
        self.pad = [_cport, local]
        self.size = 8
        self.operation = bytearray(2048)
    
    def payload(self, payload):        
        self.operation[self.size:] = bytearray(payload)
        self.size += len(payload)

    def generate(self, pad = 0):
        if self.size % 4 != 0 and pad:
            self.operation[self.size:] = [0]*(4 - (self.size % 4))
            self.size += (4 - (self.size % 4))
        struct.pack_into(GBOperation.OPERATION_HEADER_FMT, self.operation, 0,
                 self.size, self.id, self.type, self.result,
                 self.pad[0], self.pad[1])
        return self.operation[:self.size]
    


In [ ]:
class GBControlOp(object):
    GB_CONTROL_TYPE_PROTOCOL_VERSION = 0x01
    GB_CONTROL_TYPE_GET_MANIFEST_SIZE = 0x03
    GB_CONTROL_TYPE_GET_MANIFEST = 0x04
    GB_CONTROL_TYPE_GET_GLOBAL_SCRATCH = 0x32
    def __init__(self, cport = 0):
        self.cport = cport

    def get_protocol_version(self):
        return GBOperation(GBControlOp.GB_CONTROL_TYPE_PROTOCOL_VERSION, self.cport, 1).generate()
    def get_manifest_size(self):
        return GBOperation(GBControlOp.GB_CONTROL_TYPE_GET_MANIFEST_SIZE, self.cport, 1).generate()
    def get_manifest(self):
        return GBOperation(GBControlOp.GB_CONTROL_TYPE_GET_MANIFEST, self.cport, 1).generate()
    def get_global_scratch(self):
        return GBOperation(GBControlOp.GB_CONTROL_TYPE_GET_GLOBAL_SCRATCH, self.cport, 1).generate()

In [ ]:
class GBI2COp(object):
    GB_I2C_PROTOCOL_VERSION = 0x01
    GB_I2C_PROTOCOL_FUNCTIONALITY = 0x02
    GB_I2C_PROTOCOL_TRANSFER = 0x05
    
    def __init__(self, cport = 2):
        self.cport = cport

    def get_protocol_version(self):
        return GBOperation(GBI2COp.GB_I2C_PROTOCOL_VERSION, self.cport, 1).generate()
    def get_functionality(self):
        return GBOperation(GBI2COp.GB_I2C_PROTOCOL_FUNCTIONALITY, self.cport, 1).generate()
    def readWord(self, address, commandCode):
        i2crwop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        i2crwop.payload([2, 0]) #number of transactions
        i2crwop.payload([address, 0, 0, 0, 1, 0])
        i2crwop.payload([address, 0, 1, 0, 2, 0])
        i2crwop.payload([commandCode])
        return i2crwop.generate()
    def readByte(self, address, commandCode):
        i2crbop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        i2crbop.payload([2, 0]) #number of transactions
        i2crbop.payload([address, 0, 0, 0, 1, 0])
        i2crbop.payload([address, 0, 1, 0, 1, 0])
        i2crbop.payload([commandCode])
        return i2crbop.generate()
    def readBlock(self, address, commandCode, nbytes):
        i2crbop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        i2crbop.payload([2, 0]) #number of transactions
        i2crbop.payload([address, 0, 0, 0, 1, 0])
        i2crbop.payload([address, 0, 1, 0, nbytes, 0])
        i2crbop.payload([commandCode])
        return i2crbop.generate()
    def writeCommand(self, address, commandCode):
        i2cwcop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        i2cwcop.payload([1, 0]) #number of transactions
        i2cwcop.payload([address, 0, 0, 0, 1, 0])
        i2cwcop.payload([commandCode])
        return i2cwcop.generate()
    def writeByte(self, address, commandCode, wbyte, split = 0):
        i2cwbop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        if split:
            i2cwbop.payload([2, 0]) #number of transactions
            i2cwbop.payload([address, 0, 0, 0, 1, 0])
            i2cwbop.payload([address, 0, 0, 0, 1, 0])
            i2cwbop.payload([commandCode])
            i2cwbop.payload([wbyte])
        else:
            i2cwbop.payload([1, 0]) #number of transactions
            i2cwbop.payload([address, 0, 0, 0, 2, 0])
            i2cwbop.payload([commandCode, wbyte])
        return i2cwbop.generate()
    def writeWord(self, address, commandCode, wword, split = 0):
        i2cwwop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        if split:
            i2cwwop.payload([2, 0]) #number of transactions
            i2cwwop.payload([address, 0, 0, 0, 1, 0])
            i2cwwop.payload([address, 0, 0, 0, 2, 0])
            i2cwwop.payload([commandCode])
            i2cwwop.payload(list(wword.to_bytes(2, 'little')))
        else:
            i2cwwop.payload([1, 0]) #number of transactions
            i2cwwop.payload([address, 0, 0, 0, 3, 0])
            i2cwwop.payload([commandCode])
            i2cwwop.payload(list(wword.to_bytes(2, 'little')))
        return i2cwwop.generate()
    def writeBlock(self, address, commandCode, block, split = 0):
        i2cwwop = GBOperation(GBI2COp.GB_I2C_PROTOCOL_TRANSFER, self.cport, 1)
        if split:
            i2cwwop.payload([2, 0]) #number of transactions
            i2cwwop.payload([address, 0, 0, 0, 1, 0])
            i2cwwop.payload([address, 0, 0, 0, len(block), 0])
            i2cwwop.payload([commandCode])
            i2cwwop.payload(block)
        else:
            i2cwwop.payload([1, 0]) #number of transactions
            i2cwwop.payload([address, 0, 0, 0, 1 + len(block), 0])
            i2cwwop.payload([commandCode])
            i2cwwop.payload(block)            
        return i2cwwop.generate()

In [ ]:
class GBLocalPlaybackOp(object):
    GB_LOCALPLAYBACK = 0xB0
    
    def __init__(self, cport = 0):
        self.cport = cport
        self.operation = GBOperation(GBLocalPlaybackOp.GB_LOCALPLAYBACK, self.cport, 0, 0xA0)
        self.opcount = 0
        self.payload = []

    def addop(self, operation):        
        self.payload.append(operation)
        self.opcount += 1
    
    def generate_lp(self):
        self.operation.payload([self.opcount, 0])
        for i in range(self.opcount):
            self.operation.payload(self.payload[i])
        return self.operation.generate()        

In [ ]:
class GBLocalIFOp(object):
    GB_LOCALIFOP = 0xC0
    
    def __init__(self, cond = 0, cport = 0):
        self.cport = cport
        self.operation = GBOperation(GBLocalIFOp.GB_LOCALIFOP, self.cport, 0, 0xA0)
        self.opcount = 0
        self.cond = cond
        self.payload = []

    def addop(self, operation):        
        self.payload.append(operation)
        self.opcount += 1
    
    def generate_lp(self):
        #FIXME: do this only once 
        self.operation.payload([self.cond, self.opcount])
        for i in range(self.opcount):
            self.operation.payload(self.payload[i])
        return self.operation.generate()

In [ ]:
class GBLocalWhileOp(object):
    GB_LOCALWHILEOP = 0xD0
    
    def __init__(self, cond = 0, cport = 0):
        self.cport = 0
        self.operation = GBOperation(GBLocalWhileOp.GB_LOCALWHILEOP, self.cport, 0, 0xA0)
        self.opcount = 0
        self.cond = cond
        self.payload = []

    def addop(self, operation):        
        self.payload.append(operation)
        self.opcount += 1
    
    def generate_lp(self):
        #FIXME: do this only once 
        self.operation.payload([self.cond, self.opcount])
        for i in range(self.opcount):
            self.operation.payload(self.payload[i])
        return self.operation.generate() 

In [ ]:
class GBLocalScratchWriteOp(object):
    GB_SCRATCHWRITE = 0xB1
    
    def __init__(self, offset = 0, value = 0, cport = 0):
        self.cport = cport
        self.operation = GBOperation(GBLocalScratchWriteOp.GB_SCRATCHWRITE, self.cport, 0, 0xA0)
        self.opcount = 0
        self.offset = offset
        self.value =  value
        self.payload = []

    def addop(self, operation):        
        self.payload.append(operation)
        self.opcount += 1
    
    def generate(self):
        self.operation.payload([self.offset, self.opcount] + list(self.value.to_bytes(4, 'little')))
        for i in range(self.opcount):
            self.operation.payload(self.payload[i])
        return self.operation.generate()

In [ ]:
class GBLocalMathOp(object):
    GB_MATHOP = 0xB2
    GB_MATH_ADD = 1
    GB_MATH_SUB = 2
    GB_MATH_MUL = 3
    GB_MATH_DIV = 4
    GB_MATH_MOD = 5
    GB_MATH_INC = 6
    GB_MATH_DEC = 7
    GB_MATH_BWAND = 8
    GB_MATH_BWOR = 9
    GB_MATH_BWXOR = 10
    GB_MATH_BWLSH = 11
    GB_MATH_BWRSH = 12
    GB_MATH_BWNOT = 13
    GB_MATH_COND_EQ = 14
    GB_MATH_COND_NE = 15
    GB_MATH_COND_GT = 16
    GB_MATH_COND_LT = 17
    GB_MATH_COND_GTEQ = 18
    GB_MATH_COND_LTEQ = 19

    def __init__(self, dest = 0, src1 = 0, src2 = 0, operand = "+", cport = 0):

        self.cport = cport
        self.operation = GBOperation(GBLocalMathOp.GB_MATHOP, self.cport, 0, 0xA0)
        self.dest = dest
        self.src1 = src1
        self.src2 = src2
        self.operand = 0
        if operand == "+":
            self.operand = GBLocalMathOp.GB_MATH_ADD
        elif operand == "-":
            self.operand = GBLocalMathOp.GB_MATH_SUB
        elif operand == "*":
            self.operand = GBLocalMathOp.GB_MATH_MUL
        elif operand == "/":
            self.operand = GBLocalMathOp.GB_MATH_DIV
        elif operand == "%":
            self.operand = GBLocalMathOp.GB_MATH_MOD
        elif operand == "++":
            self.operand = GBLocalMathOp.GB_MATH_INC
        elif operand == "--":
            self.operand = GBLocalMathOp.GB_MATH_DEC
        elif operand == "&":
            self.operand = GBLocalMathOp.GB_MATH_BWAND
        elif operand == "|":
            self.operand = GBLocalMathOp.GB_MATH_BWOR
        elif operand == "^":
            self.operand = GBLocalMathOp.GB_MATH_BWXOR
        elif operand == "<<":
            self.operand = GBLocalMathOp.GB_MATH_BWLSH
        elif operand == ">>":
            self.operand = GBLocalMathOp.GB_MATH_BWRSH
        elif operand == "-":
            self.operand = GBLocalMathOp.GB_MATH_BWNOT
        elif operand == "==":
            self.operand = GBLocalMathOp.GB_MATH_COND_EQ
        elif operand == "!=":
            self.operand = GBLocalMathOp.GB_MATH_COND_NE
        elif operand == ">":
            self.operand = GBLocalMathOp.GB_MATH_COND_GT
        elif operand == "<":
            self.operand = GBLocalMathOp.GB_MATH_COND_LT
        elif operand == ">=":
            self.operand = GBLocalMathOp.GB_MATH_COND_GTEQ
        elif operand == "<=":
            self.operand = GBLocalMathOp.GB_MATH_COND_LTEQ

    def generate(self):
        self.operation.payload([self.dest, self.src1, self.src2, self.operand])
        return self.operation.generate()

In [ ]:
class GBLocalDelayOp(object):
    GB_DELAYOP = 0xB3
    def __init__(self, delayus = 0):
        self.operation = GBOperation(GBLocalDelayOp.GB_DELAYOP,delayus%256, 0 , delayus//256)
    def generate(self):
        return self.operation.generate()

In [ ]:
class GBGPIOOp(object):
    GB_GPIO_TYPE_PROTOCOL_VERSION = 0x1
    GB_GPIO_TYPE_DIRECTION_IN = 0x06
    GB_GPIO_TYPE_DIRECTION_OUT = 0x07
    GB_GPIO_TYPE_GET_VALUE = 0x08
    GB_GPIO_TYPE_SET_VALUE = 0x09
    
    def __init__(self, cport = 1):
        self.cport = cport

    def get_protocol_version(self):
        return GBOperation(GBGPIOOp.GB_GPIO_TYPE_PROTOCOL_VERSION, self.cport, 1).generate()
    def direction_in(self, which):
        gpiodiroutop = GBOperation(GBGPIOOp.GB_GPIO_TYPE_DIRECTION_IN, self.cport, 1)
        gpiodiroutop.payload([which])
        return gpiodiroutop.generate()
    def direction_out(self, which, value):
        gpiodiroutop = GBOperation(GBGPIOOp.GB_GPIO_TYPE_DIRECTION_OUT, self.cport, 1)
        gpiodiroutop.payload([which, value])
        return gpiodiroutop.generate()
    def getvalue(self, which):
        gpiogv = GBOperation(GBGPIOOp.GB_GPIO_TYPE_GET_VALUE, self.cport, 1)
        gpiogv.payload([which])
        return gpiogv.generate()
    def setvalue(self, which, value):
        gpiosv = GBOperation(GBGPIOOp.GB_GPIO_TYPE_SET_VALUE, self.cport, 1)
        gpiosv.payload([which, value])
        return gpiosv.generate()


In [ ]:
class GBGPIO(object):
    def __init__(self, transport, cport):
        self.cport = cport
        self.transport = transport
    def direction_out(self, which, value):        
        self.transport.write(GBGPIOOp().direction_out(which, value), self.cport)
        return self.transport.read(8, self.cport)[5]
    def direction_in(self, which):
        self.transport.write(GBGPIOOp().direction_in(which), self.cport)
        return self.transport.read(8, self.cport)[5]
    def setvalue(self, which, value):
        self.transport.write(GBGPIOOp().setvalue(which, value), self.cport)
        return self.transport.read(8, self.cport)[5]
    def getvalue(self, which):
        self.transport.write(GBGPIOOp().getvalue(which), self.cport)
        return self.transport.read(9, self.cport)[8]

class GBI2C(object):
    def __init__(self, transport, cport):
        self.cport = cport
        self.transport = transport
    def ReadByte(self, address, command):
        self.transport.write(GBI2COp(self.cport).readByte(address, command), self.cport)
        return self.transport.read(9, self.cport)[8]
    def ReadWord(self, address, command):
        self.transport.write(GBI2COp(self.cport).readWord(address, command), self.cport)
        return int.from_bytes(self.transport.read(10, self.cport)[8:10], 'little')
    def ReadBlock(self, address, command, nbytes):
        self.transport.write(GBI2COp(self.cport).readBlock(address, command, nbytes), self.cport)
        return list(self.transport.read(8 + nbytes, self.cport)[8:])
    def WriteCommand(self, address, command):
        self.transport.write(GBI2COp(self.cport).writeCommand(address, command), self.cport)
        return self.transport.read(8, self.cport)[5]
    def WriteByte(self, address, command, wrbyte):
        self.transport.write(GBI2COp(self.cport).writeByte(address, command, wrbyte), self.cport)
        return self.transport.read(8, self.cport)[5]
    def WriteWord(self, address, command, word):
        self.transport.write(GBI2COp(self.cport).writeWord(address, command, word), self.cport)
        return self.transport.read(8, self.cport)[5]
    def WriteBlock(self, address, command, block):
        self.transport.write(GBI2COp(self.cport).writeBlock(address, command, block), self.cport)
        return self.transport.read(8, self.cport)[5]

class GBControl(object):
    def __init__(self, transport, cport):
        self.cport = cport
        self.transport = transport
    def get_manifest(self):
        get_mnfs_size = GBControlOp(self.cport).get_manifest_size()
        self.transport.write(get_mnfs_size, self.cport)
        mnfssize = int.from_bytes(self.transport.read(10)[8:9], "little")
        print("manifest is :" + str(mnfssize), self.cport)
        get_manifest = GBControlOp(self.cport).get_manifest()
        self.transport.write(get_manifest, self.cport)
        return self.transport.read(8 + mnfssize)[8:]
    def get_scratch_all(self):
        get_global_scratch = GBControlOp(self.cport).get_global_scratch()
        self.transport.write(get_global_scratch, self.cport)
        scratchbuffer = list(self.transport.read(136, self.cport)[8:])
        scratch = []
        for i in range(0, len(scratchbuffer), 4):
            scratch.append(int.from_bytes(scratchbuffer[i:(i+3)], 'little'))
        return scratch
    def get_scratch(self, offset=0):
        get_global_scratch = GBControlOp(self.cport).get_global_scratch()
        self.transport.write(get_global_scratch, self.cport)
        scratchbuffer = list(self.transport.read(136, self.cport)[8:])
        scratch = []
        for i in range(0, len(scratchbuffer), 4):
            scratch.append(int.from_bytes(scratchbuffer[i:(i+3)], 'little'))
        return scratch[offset]
    def scratchwrite(self, offset, value):
        lpop = GBLocalPlaybackOp()
        scop = GBLocalScratchWriteOp(offset, value)
        lpop.addop(scop.generate())
        self.transport.write(lpop.generate_lp(), self.cport)
    def mathop(self, dest, src1, src2, operator):
        lpop = GBLocalPlaybackOp()
        mathop = GBLocalMathOp(dest, src1, src2, operator)
        lpop.addop(mathop.generate())
        self.transport.write(lpop.generate_lp(), self.cport)
        

class GBModule(object):    
    def __init__(self, transport, control = 0, gpio = 1, i2c = 2):
        self.Control = GBControl(transport, control)
        #TODO parse manifest
        self.GPIO = GBGPIO(transport, gpio)
        self.I2C = GBI2C(transport, i2c)
        self.transport = transport

In [ ]:
class SerialTransport(object):
    def __init__(self, port, baud = 115200):
        self.transport = serial.Serial(port, baud)
    
    def read(self, count, cport = 0):
        #read header
        header = list(self.transport.read(8))
        size = int.from_bytes(header[0:1], 'little')
        if size > 0:
            header = header + list(self.transport.read(size - 8))
        if size != count:
            print("invalid packet received")
            print(size, header)
            header = list(self.transport.read(8))
            size = int.from_bytes(header[0:1], 'little')
            if size > 0:
                header = header + list(self.transport.read(size - 8))
        return bytes(header)
    def write(self, buf, cport = 0):
        self.transport.write(buf)

In [ ]:
class TCPIPV6Transport(object):
    def __init__(self, address, control = 4242, gpio = 4243, i2c = 4244):
        controlc = socket.socket(socket.AF_INET6, socket.SOCK_STREAM, 0)
        gpioc = socket.socket(socket.AF_INET6, socket.SOCK_STREAM, 0)
        i2cc = socket.socket(socket.AF_INET6, socket.SOCK_STREAM, 0)
        self.transport = [controlc, gpioc, i2cc]
        controlc.connect((address, control, 0, 0))
        gpioc.connect((address, gpio, 0, 0))
        i2cc.connect((address, i2c, 0, 0))

    def read(self, count, cport = 0):
        #read header
        header = list(self.transport[cport].recv(8))
        size = int.from_bytes(header[0:1], 'little')
        if size > 0:
            header = header + list(self.transport[cport].recv(size - 8))
        if size != count:
            print("invalid packet received")
            print(size, header)
            header = list(self.transport[cport].recv(8))
            size = int.from_bytes(header[0:1], 'little')
            if size > 0:
                header = header + list(self.transport[cport].recv(size - 8))
        return bytes(header)
    def write(self, buf, cport = 0):
        self.transport[cport].sendall(buf)

In [ ]:
ipv6transport = TCPIPV6Transport("2001:db8::1")
beagleconnect = GBModule(ipv6transport)
bcfcontrol = beagleconnect.Control
bcfgpio = beagleconnect.GPIO
bcfi2c = beagleconnect.I2C

In [ ]:
print(bcfcontrol.get_manifest())
# print(bcfcontrol.get_scratch())
# print(bcfi2c.ReadByte(0x68, 0))
# print(bcfi2c.ReadWord(0x68, 0))
# print(bcfi2c.ReadBlock(0x68, 0, 12))
# print(bcfi2c.WriteCommand(0x68, 0))
# print(bcfi2c.WriteByte(0x68, 0x40, 0x44))
# print(bcfi2c.WriteWord(0x68, 0x40, 0x44))
# print(bcfi2c.WriteBlock(0x68, 0x40, [0x41, 0x42, 0x43, 0x44]))
# print(bcfgpio.direction_out(0,1))
# print(bcfgpio.direction_in(0))
# print(bcfgpio.setvalue(0,1))
# print(bcfgpio.getvalue(0))

In [ ]:
print(bcfcontrol.get_scratch(0))
bcfcontrol.scratchwrite(0, 5)
print(bcfcontrol.get_scratch(0))

In [ ]:
lpop = GBLocalPlaybackOp()
get_mnfs_version = GBControlOp().get_protocol_version()
get_manifest = GBControlOp().get_manifest()
lpop.addop(get_mnfs_version)
lpop.addop(get_manifest)
mylpop = lpop.generate_lp()
print("".join("0x%02x " % b for b in mylpop))
beagleconnect.transport.write(mylpop)

In [ ]:
lpop = GBLocalPlaybackOp()
delayop = GBLocalDelayOp(2000)
lpop.addop(delayop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
#scratchwritepayload from another operation
lpop = GBLocalPlaybackOp()
scop = GBLocalScratchWriteOp(2, 0) #offset, #value
scop.addop(GBI2COp(2).readByte(0x68, 0))
lpop.addop(scop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
#scratchwrite constant
lpop = GBLocalPlaybackOp()
scop = GBLocalScratchWriteOp(0, 2) #offset, #value
lpop.addop(scop.generate())
beagleconnect.transport.write(lpop.generate_lp())

lpop = GBLocalPlaybackOp()
scop = GBLocalScratchWriteOp(1, 3) #offset, #value
lpop.addop(scop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
bcfcontrol.scratchwrite(5, 1)
ifop = GBLocalIFOp(5)
ifop.addop(GBControlOp().get_manifest())
ifop.addop(GBLocalMathOp(5, 0, 0, "--").generate())
# beagleconnect.transport.write(ifop.generate_lp())

lpop = GBLocalPlaybackOp()
mathop = GBLocalMathOp(0, 0, 0, "+")
lpop.addop(mathop.generate())
lpop.addop(ifop.generate_lp())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
# # start infinite playback
lpop = GBLocalPlaybackOp(0xAA)
mathop = GBLocalMathOp(0, 0, 0, "++")
lpop.addop(mathop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
# # start infinite playback
lpop = GBLocalPlaybackOp(0xAA)
mathop = GBLocalMathOp(1, 0, 0, "++")
lpop.addop(mathop.generate())

ifop = GBLocalIFOp(5)
ifop.addop(GBControlOp().get_manifest())
ifop.addop(GBLocalMathOp(5, 0, 0, "--").generate())
lpop.addop(ifop.generate_lp())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
# # stop infinite playback
lpop = GBLocalPlaybackOp(0xA1)
mathop = GBLocalMathOp(0, 0, 0, "+")
lpop.addop(mathop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
#ifop
bcfcontrol.scratchwrite(5, 1)
ifop = GBLocalIFOp(5)
ifop.addop(GBControlOp().get_manifest())
ifop.addop(GBLocalMathOp(5, 0, 0, "--").generate())
beagleconnect.transport.write(ifop.generate_lp())

In [ ]:
#scratchwritepayload from another operation
lpop = GBLocalPlaybackOp()
scop = GBLocalScratchWriteOp(2, 0) #offset, #value
scop.addop(GBI2COp(2).readByte(0x68, 0))
lpop.addop(scop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
#whileop
bcfcontrol.scratchwrite(5, 5)
whileop = GBLocalWhileOp(5)
whileop.addop(GBControlOp().get_manifest())
whileop.addop(GBLocalMathOp(5, 0, 0, "--").generate())
beagleconnect.transport.write(whileop.generate_lp())

In [ ]:
print(GBLocalMathOp(5, 0, 0, "--").generate())

In [ ]:
print(bcfgpio.direction_out(18,1))
# print(bcfgpio.direction_in(0))

# print(bcfgpio.getvalue(0))

In [ ]:
print(bcfgpio.setvalue(18,1))

In [ ]:
import datetime
print(datetime.datetime.now())
for i in range(100):
         bcfgpio.setvalue(18, 0)
         bcfgpio.setvalue(18, 1)
print(datetime.datetime.now())

In [ ]:
class OPT3001(object):
    def __init__(self, bus, address):
        self.bus = bus
        self.address = address
        print("mfrid = " + str(hex(bus.ReadWord(address, 0x7E))))
        print("device ID = " + str(hex(bus.ReadWord(address, 0x7F))))
        cfgread = self.read_register_16bit(1)
        cfgset = cfgread | (3 << 9)
        self.write_config_reg(cfgset)

    def read_register_16bit(self, adr):
        values = self.bus.ReadBlock(self.address, adr, 2)
        data = (values[0] << 8) | values[1]
        return data

    def write_register_16bit(self, adr, data):
        d1 = (data >> 8)
        d0 = data & 0xFF
        return self.bus.WriteBlock(self.address, adr, [d1, d0])

    def write_config_reg(self, data):
        return self.write_register_16bit(1, data)

    def read_lux_float(self):
        req_value = self.read_register_16bit(0)
        mantisse = req_value & 0x0fff
        exponent = (req_value & 0xf000) >> 12
        return 2**exponent * mantisse * 0.01

In [ ]:
class HDC2010(object):
    def __init__(self, bus, address, auto= 0):
        self.bus = bus
        self.address = address
        print("mfrid = " + str(hex(bus.ReadWord(address, 0xFC))))
        print("device ID = " + str(hex(bus.ReadWord(address, 0xFE))))
        bus.WriteByte(address, 0x0E, 0x80)
        sleep(0.001)
    
    def readSensor(self):
        self.bus.WriteBlock(self.address, 0x0F, [0x01])
        sleep(0.005)
        val = self.bus.ReadBlock(self.address, 0x00, 4)
        print(val)
        temp = val[1]*256 + val[0]
        temperature = (float(temp) * 165)/65536.0 - 40
        hum = val[3]*256 + val[2]
        relhum = (float(hum) * 100)/65536.0
        print(temperature, relhum)
        return (temperature, relhum)

In [ ]:
temphum = HDC2010(bcfi2c, 0x41)

In [ ]:
print(temphum.readSensor())

In [ ]:
light = OPT3001(bcfi2c, 0x44)

In [ ]:
print(light.read_lux_float())

In [ ]:
#scratchwritepayload from another operation
lpop = GBLocalPlaybackOp(0xAA)
scop = GBLocalScratchWriteOp(0, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(5))
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(1, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(27))
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(2, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(10))
lpop.addop(scop.generate()) 
scop = GBLocalScratchWriteOp(3, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(11))
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(4, 0)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(5, 1)
lpop.addop(scop.generate())
lpop.addop(GBLocalMathOp(4, 4, 0, "+").generate())
lpop.addop(GBLocalMathOp(4, 4, 5, "<<").generate())
lpop.addop(GBLocalMathOp(4, 4, 1, "+").generate())
lpop.addop(GBLocalMathOp(4, 4, 5, "<<").generate())
lpop.addop(GBLocalMathOp(4, 4, 2, "+").generate())
lpop.addop(GBLocalMathOp(4, 4, 5, "<<").generate())
lpop.addop(GBLocalMathOp(4, 4, 3, "+").generate())
scop = GBLocalScratchWriteOp(5, 1)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(6, 3)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(7, 2)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(8, 6)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(9, 4)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(10, 12)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(11, 8)
lpop.addop(scop.generate())
beagleconnect.transport.write(lpop.generate_lp())

In [ ]:
# # stop infinite playback
lpop = GBLocalPlaybackOp(0xA1)
mathop = GBLocalMathOp(0, 0, 0, "+")
lpop.addop(mathop.generate())
beagleconnect.transport.write(lpop.generate_lp())
print(bcfgpio.direction_out(19,1))
print(bcfgpio.direction_out(20,1))
print(bcfgpio.direction_out(21,1))
print(bcfgpio.direction_out(22,1))

In [ ]:
# Registers/etc:
PCA9685_ADDRESS    = 0x40
MODE1              = 0x00
MODE2              = 0x01
SUBADR1            = 0x02
SUBADR2            = 0x03
SUBADR3            = 0x04
PRESCALE           = 0xFE
LED0_ON_L          = 0x06
LED0_ON_H          = 0x07
LED0_OFF_L         = 0x08
LED0_OFF_H         = 0x09
ALL_LED_ON_L       = 0xFA
ALL_LED_ON_H       = 0xFB
ALL_LED_OFF_L      = 0xFC
ALL_LED_OFF_H      = 0xFD

# Bits:
RESTART            = 0x80
SLEEP              = 0x10
ALLCALL            = 0x01
INVRT              = 0x10
OUTDRV             = 0x04
import math
class PCA9685(object):
    """PCA9685 PWM LED/servo controller."""

    def __init__(self,i2c=None, address=PCA9685_ADDRESS):
        self.i2c = i2c
        self.address = address
        mode1 = i2c.ReadByte(self.address, MODE1)
        print(mode1)
        i2c.WriteByte(self.address, MODE1, 0)
        sleep(0.005)

    def set_pwm_freq(self, freq_hz):
        prescaleval = 25000000.0    # 25MHz
        prescaleval /= 4096.0       # 12-bit
        prescaleval /= float(freq_hz)
        prescaleval -= 1.0
        prescale = int(math.floor(prescaleval + 0.5))
        oldmode = self.i2c.ReadByte(self.address, MODE1);
        newmode = (oldmode & 0x7F) | 0x10    # sleep
        self.i2c.WriteByte(self.address, MODE1, newmode)  # go to sleep
        self.i2c.WriteByte(self.address, PRESCALE, prescale)
        self.i2c.WriteByte(self.address, MODE1, oldmode)
        sleep(0.005)
        self.i2c.WriteByte(self.address, MODE1, oldmode | 0xA0)

    def set_pwm(self, channel, on, off):
        self.i2c.WriteByte(self.address, LED0_ON_L+4*channel, on & 0xFF)
        self.i2c.WriteByte(self.address, LED0_ON_H+4*channel, on >> 8)
        self.i2c.WriteByte(self.address, LED0_OFF_L+4*channel, off & 0xFF)
        self.i2c.WriteByte(self.address, LED0_OFF_H+4*channel, off >> 8)
    
    def map_range(self, x, in_min, in_max, out_min, out_max):
        return (x - in_min) * (out_max - out_min) // (in_max - in_min) + out_min

    def set_duty(self, channel, duty):
        if (duty == 100):
            self.set_pwm(channel, 0x1000, 0)
        elif (duty == 0):
            self.set_pwm(channel, 0, 0x1000)
        else:
            duty = self.map_range(duty, 0, 100, 0xa00, 0xFFE)
#             print("setting " + str((duty)))
            self.set_pwm(channel, 0, duty)
        
        
    def set_all_pwm(self, on, off):
        self.i2c.WriteByte(self.address, ALL_LED_ON_L, on & 0xFF)
        self.i2c.WriteByte(self.address, ALL_LED_ON_H, on >> 8)
        self.i2c.WriteByte(self.address, ALL_LED_OFF_L, off & 0xFF)
        self.i2c.WriteByte(self.address, ALL_LED_OFF_H, off >> 8)

In [ ]:
pwm = PCA9685(bcfi2c, 0x40)

In [ ]:
# pwm.set_all_pwm(0x0, 0xb00)

In [ ]:
pwm.set_pwm_freq(10000)

In [ ]:
pwm.set_all_pwm(0, 0)

In [ ]:
# setup gpio as input
print(bcfgpio.direction_in(5))
print(bcfgpio.direction_in(27))
print(bcfgpio.direction_in(10))
print(bcfgpio.direction_in(11))
# motor control
print(bcfgpio.direction_out(19,1))
print(bcfgpio.direction_out(20,1))
print(bcfgpio.direction_out(21,1))
print(bcfgpio.direction_out(22,1))
pwm.set_duty(0, 10)
pwm.set_duty(1, 10)
print(bcfgpio.setvalue(19,1))
print(bcfgpio.setvalue(20,0))
print(bcfgpio.setvalue(21,1))
print(bcfgpio.setvalue(22,0))

In [ ]:
# reverse motor
print(bcfgpio.setvalue(19,0))
print(bcfgpio.setvalue(20,1))
print(bcfgpio.setvalue(21,0))
print(bcfgpio.setvalue(22,1))

In [ ]:
#scratchwritepayload from another operation
lpop = GBLocalPlaybackOp(0xAA)
scop = GBLocalScratchWriteOp(0, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(5))
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(1, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(27))
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(2, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(10))
lpop.addop(scop.generate()) 
scop = GBLocalScratchWriteOp(3, 0) #offset, #value
scop.addop(GBGPIOOp().getvalue(11))
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(4, 0)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(5, 1)
lpop.addop(scop.generate())
lpop.addop(GBLocalMathOp(4, 4, 0, "+").generate())
lpop.addop(GBLocalMathOp(4, 4, 5, "<<").generate())
lpop.addop(GBLocalMathOp(4, 4, 1, "+").generate())
lpop.addop(GBLocalMathOp(4, 4, 5, "<<").generate())
lpop.addop(GBLocalMathOp(4, 4, 2, "+").generate())
lpop.addop(GBLocalMathOp(4, 4, 5, "<<").generate())
lpop.addop(GBLocalMathOp(4, 4, 3, "+").generate())
scop = GBLocalScratchWriteOp(5, 1)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(6, 3)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(7, 2)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(8, 6)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(9, 4)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(10, 12)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(11, 8)
lpop.addop(scop.generate())
lpop.addop(GBLocalMathOp(12, 4, 5, "==").generate())
lpop.addop(GBLocalMathOp(13, 4, 6, "==").generate())
lpop.addop(GBLocalMathOp(14, 4, 7, "==").generate())
lpop.addop(GBLocalMathOp(15, 4, 8, "==").generate())
lpop.addop(GBLocalMathOp(16, 4, 9, "==").generate())
lpop.addop(GBLocalMathOp(17, 4, 10, "==").generate())
lpop.addop(GBLocalMathOp(18, 4, 11, "==").generate())
scop = GBLocalScratchWriteOp(19, 1)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(20, 2)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(21, 3)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(22, 4)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(23, 5)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(24, 0)
lpop.addop(scop.generate())
lpop.addop(GBLocalMathOp(24, 12, 19, "*").generate())
lpop.addop(GBLocalMathOp(25, 13, 20, "*").generate())
lpop.addop(GBLocalMathOp(24, 24, 25, "+").generate())
lpop.addop(GBLocalMathOp(25, 14, 21, "*").generate())
lpop.addop(GBLocalMathOp(24, 24, 25, "+").generate())
lpop.addop(GBLocalMathOp(25, 15, 21, "*").generate())
lpop.addop(GBLocalMathOp(24, 24, 25, "+").generate())
lpop.addop(GBLocalMathOp(25, 16, 21, "*").generate())
lpop.addop(GBLocalMathOp(24, 24, 25, "+").generate())
lpop.addop(GBLocalMathOp(25, 17, 22, "*").generate())
lpop.addop(GBLocalMathOp(24, 24, 25, "+").generate())
lpop.addop(GBLocalMathOp(25, 18, 23, "*").generate())
lpop.addop(GBLocalMathOp(24, 24, 25, "+").generate())
scop = GBLocalScratchWriteOp(26, 0)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(27, 0)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(28, 2)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(29, 4)
lpop.addop(scop.generate())
scop = GBLocalScratchWriteOp(30, 0)
lpop.addop(scop.generate())
lpop.addop(GBLocalMathOp(26, 24, 28, "==").generate())
lpop.addop(GBLocalMathOp(27, 24, 29, "==").generate())
# lpop.addop(GBLocalDelayOp(1000).generate())

ifop = GBLocalIFOp(26) 
ifop.addop(GBGPIOOp().setvalue(19, 0))
ifop.addop(GBGPIOOp().setvalue(20, 0))
ifop.addop(GBGPIOOp().setvalue(21, 1))
ifop.addop(GBGPIOOp().setvalue(22, 0))
lpop.addop(ifop.generate_lp())
ifop = GBLocalIFOp(27) 
ifop.addop(GBGPIOOp().setvalue(19, 1))
ifop.addop(GBGPIOOp().setvalue(20, 0))
ifop.addop(GBGPIOOp().setvalue(21, 0))
ifop.addop(GBGPIOOp().setvalue(22, 0))
# beagleconnect.transport.write(ifop.generate_lp())
# beagleconnect.transport.write(ifop.generate_lp())
lpop.addop(ifop.generate_lp())
# print(len(lpop.generate_lp()))
beagleconnect.transport.write(lpop.generate_lp())


In [ ]:
print(bcfcontrol.get_scratch(26))
print(bcfcontrol.get_scratch(27))

In [ ]:
# # stop infinite playback
lpop = GBLocalPlaybackOp(0xA1)
mathop = GBLocalMathOp(0, 0, 0, "+")
lpop.addop(mathop.generate())
beagleconnect.transport.write(lpop.generate_lp())
print(bcfgpio.direction_out(19,1))
print(bcfgpio.direction_out(20,1))
print(bcfgpio.direction_out(21,1))
print(bcfgpio.direction_out(22,1))

In [ ]:
def update_speed(speed):
    # setup gpio as input
    print(bcfgpio.direction_in(5))
    print(bcfgpio.direction_in(27))
    print(bcfgpio.direction_in(10))
    print(bcfgpio.direction_in(11))
    # motor control
    print(bcfgpio.direction_out(19,1))
    print(bcfgpio.direction_out(20,1))
    print(bcfgpio.direction_out(21,1))
    print(bcfgpio.direction_out(22,1))
    pwm.set_duty(0, speed)
    pwm.set_duty(1, speed)
    print(bcfgpio.setvalue(19,1))
    print(bcfgpio.setvalue(20,0))
    print(bcfgpio.setvalue(21,1))
    print(bcfgpio.setvalue(22,0))

In [ ]:
update_speed(100)